In [ ]:
!git clone https://github.com/nishantkushwaha-1999/llm_fine_tuning_optimization.git

Cloning into 'llm_fine_tuning_optimization'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 48 (delta 19), reused 35 (delta 9), pack-reused 0
Receiving objects: 100% (48/48), 6.31 MiB | 14.69 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
!pip install -r ./llm_fine_tuning_optimization/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uni

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from llm_fine_tuning_optimization.Tunner import fine_tunner

## Read the data

In [ ]:
df = pd.read_csv("./llm_fine_tuning_optimization/Data/train_samples.csv")[:100]
df.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [ ]:
tuner = fine_tunner.Llama_Tuner()

Mounted at /content/drive


In [ ]:
train_data = tuner.to_dataset(list(df['question']), list(df['answer']))

i = 0
for _ in train_data:
  print(_)
  i += 1
  if i >= 10:
    break

{'text': "<s>[INST] What is (are) Glaucoma ? [/INST] Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic nerve and other parts of the eye and result in loss of vision. Op

## Model

In [ ]:
model_name = 'EleutherAI/pythia-70m'
save_name = './drive/MyDrive/EleutherAI/pythia-70m'

In [ ]:

quant_config = {
    'load_in_4bit': True,
    'bnb_4bit_quant_type': "nf4",
    'bnb_4bit_compute_dtype': getattr(torch, "float16"),
    'bnb_4bit_use_double_quant': False
}

lora_config = {
    'lora_alpha': 16,
    'lora_dropout': 0.1,
    'r': 64,
    'bias': "none",
    'task_type': "CASUAL_LM"
}

train_config = {
    'output_dir': "./results",
    'num_train_epochs': 50,
    'per_device_train_batch_size': 4,
    'gradient_accumulation_steps': 1,
    'optim': "paged_adamw_32bit",
    'save_steps': 25,
    'logging_steps': 25,
    'learning_rate': 2e-4,
    'weight_decay': 0.001,
    'fp16': False,
    'bf16': False,
    'max_grad_norm': 0.3,
    'max_steps': -1,
    'warmup_ratio': 0.03,
    'group_by_length': True,
    'lr_scheduler_type': "constant",
    'report_to': "tensorboard",
    'remove_unused_columns': False
}

In [ ]:
tuner.load_model(model_name=model_name, quant_config=quant_config, lora_config=lora_config, train_config=train_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

In [ ]:
tuner.tune_and_save(train_data, save_name)

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,3.282800
50,2.912400
75,2.728500
100,2.641100
125,2.583700
150,2.547700
175,2.467700
200,2.451200
225,2.407100
250,2.383400


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [ ]:
prompt = "What is (are) Glaucoma ?"
tuner.generate_base_text(prompt)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant

'<s>[INST] What is (are) Glaucoma ? [/INST] The optic nerve is a bundle of mostly normal sized, growing, growing, and growing sized light microscopes. It can also run through a variety of tissues. It has a variety of surgical tools. </s> - Motion Analysis. </s> - Motion Analysis. </s>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</'

In [ ]:
tuner.generate_text(prompt)

The model 'PeftModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'Rober

'<s>[INST] What is (are) Glaucoma ? [/INST] Open-angle glaucoma is the most common type of glaucoma seen in the eye. It is called a "glaucoma" vision loss. Glaucoma is caused by a combination of a variety of agents, including laser-cut sawing, high-speed scanning technology, and/or high-pitched scanning technology. The eye movement is affected by a variety of factors.  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  '

In [ ]:
test_df = pd.read_csv("./llm_fine_tuning_optimization/Data/test.csv")[:2]
test_dataset = tuner.to_dataset(list(test_df['question']), list(test_df['answer']))

reference = []
for item in test_dataset:
  reference.append(item['text'])

response = []
base_response = []
for ques in tqdm(test_df['question']):
  base_resp = tuner.generate_base_text(ques)
  resp = tuner.generate_text(ques)
  base_response.append(base_resp.strip())
  response.append(resp.strip())

  base_df = pd.DataFrame(base_response, columns=['response'])
  base_df.to_csv(f"./drive/MyDrive/{model_name}.csv", index=False)

  tuned_df = pd.DataFrame(response, columns=['response'])
  tuned_df.to_csv(f"./drive/MyDrive/{model_name}_tuned.csv", index=False)

bleu_score_base = tuner.eval_model(reference, base_response, ['bleu'])
print(f"The bleu score for base model is {bleu_score_base}")

bleu_score_tuned = tuner.eval_model(reference, response, ['bleu'])
print(f"The bleu score for tuned model is {bleu_score_tuned}")

rouge_score_base = tuner.eval_model(reference, base_response, ['rouge'])
print(f"The rouge score for base model is {rouge_score_base}")

rouge_score_tuned = tuner.eval_model(reference, response, ['rouge'])
print(f"The rouge score for tuned model is {rouge_score_tuned}")

ter_score_base = tuner.eval_model(reference, base_response, ['ter'])
print(f"The ter score for base model is {ter_score_base}")

ter_score_tuned = tuner.eval_model(reference, response, ['ter'])
print(f"The ter score for tuned model is {ter_score_tuned}")

  0%|          | 0/2 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
The model 'PeftModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', '

The bleu score for base model is (0.033528395384803164,)
The bleu score for tuned model is (0.041333114185783726,)
The rouge score for base model is (0.050997782705099776,)
The rouge score for tuned model is (0.0598669623059867,)
The ter score for base model is (0.8989459815546772,)
The ter score for tuned model is (0.8992753623188405,)
